In [1]:
# https://datascience.stackexchange.com/questions/16890/neural-network-for-multiple-output-regression?newreg=834c5e00cd8e4daab0c02a341ed88a6e

In [2]:
# https://stackoverflow.com/questions/5364050/reloading-submodules-in-ipython
%load_ext autoreload
%autoreload 2

import pymongo
import pandas as pd
import numpy as np
import os.path

from pymongo import MongoClient
from processing import *
from IPython.display import Image
from IPython.display import clear_output


In [3]:
client = MongoClient('mongodb://192.168.0.104:27017/')
db = client.geometry
objects = db.objects
files = db.files

In [4]:
files.find({'Valid':True}).count()

/home/mk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


2106

In [5]:
#fileidsWithDims = objects.find({'ClassName':'AcDbRotatedDimension'}).distinct('FileId')[:5]
fileIds = files.find({
    'Valid':False,
    'Checked':{'$ne':True}
}).distinct('FileId')
len(fileIds)

19462

In [6]:
# https://stackoverflow.com/questions/13710770/how-to-update-values-using-pymongo
def set_fileId_valid(collection, fileId, valid):
    print('updating ',fileId, 'set valid:', valid)
    collection.update_many(
        {
            'FileId':fileId
        },
        {
            '$set':{
                'Valid':valid,
                'Checked':True
            }
        },
        upsert = False)

In [ ]:
for fileid in fileIds:
    valid = False
    # try:
    data = query_collection_to_dataframe(objects, fileid)
    cols_to_expand = ['XLine1Point', 'XLine2Point','StartPoint','EndPoint','Position']
    data = expand_columns(data, cols_to_expand)
    data = normalize(data, to_size = 400)
    generate_file2(data)
    
    filename = fileid+ '.png'
    
    if os.path.exists(filename):
        display(Image(filename))  
        
        # request manual imput of file assesment
        # if it is valid or not
        valid = input() == '1'
        
    set_fileId_valid(files, fileid, valid)
    
    clear_output(wait=True)